<div>
<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=220 height=220 align=left class="saturate">

<br>
<font face="Times New Roman">
<div dir=ltr align=center> 
<!-- <font color=0F5298 size=7> -->
<font color=0F5298 size=6>
    Introduction to Machine Learning <br> <br>
<!-- <font color=2565AE size=5> -->
<font size=5>
    Computer Engineering Department <br>
    Spring 2023 <br> <br>
<font color=606060 size=5>
    Homework 4: Practical - Naive Bayes <br> <br>
<font color=686880 size=4>
    TAs: Alireza Farashah - Arman Malekzadeh - Ali Salesi
    
____

### Full Name : Mehdi Lotfian
### Student Number : 99105689
### Colab Link: https://colab.research.google.com/drive/1BgKVj3v9ZNYZ4D2QOTWGd0mcEyoAX6C9?usp=sharing
___

Note 1: In this assignment, we are trying to simulate the functionality of something called "CountVectorizer" which is used in natural language processing. You are advised to take a look at this link before beginning to answer the questions:

https://www.geeksforgeeks.org/using-countvectorizer-to-extracting-features-from-text/

Note 2: One or two TA sessions will be held related to this assignment to make sure you get familiarized with this topic. Therefore, keep calm and write code!

# Download the Dataset

In [1]:
!wget -O /content/dataset.zip "https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1"

--2023-05-11 06:46:51--  https://www.dropbox.com/s/yf195wl0sp3term/dataset-train.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/yf195wl0sp3term/dataset-train.zip [following]
--2023-05-11 06:46:51--  https://www.dropbox.com/s/dl/yf195wl0sp3term/dataset-train.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd3ebd092ad73c7ecd6850c8230.dl.dropboxusercontent.com/cd/0/get/B70MpZZWRQYFU1XGoSiwaLoJsKZKJSvDkgcqqUd_mxRzKOQoACWLpiP3Ah45Soebp-ril7g5l_HRStQONAIyZx9lKX5pRDAs1uE1jSD9zsBSfrhlxOkqRvzRHKgqfDDacLfKJpmMQ0G0oLpqEf3MJvWAUFUWxFFjAUoM863RMqHQVOdjGFDMKX9g-DU4tzzofg0/file?dl=1# [following]
--2023-05-11 06:46:52--  https://ucd3ebd092ad73c7ecd6850c8230.dl.dropboxusercontent.com/cd/0/get/B70MpZZWRQYFU1XGoSiwaLoJsKZKJSvDkgcqqUd_m

In [2]:
!mkdir /content/dataset/
!unzip '/content/dataset.zip' -d /content/dataset/

Archive:  /content/dataset.zip
  inflating: /content/dataset/content/news-train.csv  


# Preprocessing

In [3]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Read the csv file and load it as a dataframe.

In [4]:
df = pd.read_csv("dataset/content/news-train.csv")
df

,topic,text
0,sports,به گزارش ورزش سه تیم فوتبال پدیده در روزی که ا...
1,politics,پاسخ مثبت به انتقادات شورای نگهبان البته # قبل...
2,politics,# دهم در پاسخ به سوالی درباره اینکه آیا سفر وی...
3,economics,کنفرانس # بودن بازار خودرو و تاثیر آن بر اقتصا...
4,economics,به گزارش ایسنا صحبت با برخی # # آن است که امرو...
...,...,...
197356,sports,به گزارش ورزش سه و به نقل از مارکا رقابت‌های ب...
197357,sports,به گزارش ایسنا و به نقل از فوتبال ایتالیا یوون...
197358,sports,سرمربی تیم والیبال دانشجویان گفت ما اصلا فرصت ...
197359,sports,به گزارش ورزش سه مرتضی # باشگاه # مصر # را نسب...


# Dealing with Class Imbalance

For each "topic", count and display the number of news belonging to it. (5 points)

In [5]:
print(df.groupby('topic').size())

topic
cultural      13359
economics     49600
politics      11297
sports       123105
dtype: int64


Balance the dataset in a way that all of the topics get associated with the same number of news. For this purpose, find the topic for which the minimum number of news exists. Then, use downsampling to lower the number of news associated with the other topics.<br>
Finally, save the result as a new dataframe. (15 points)

In [6]:
min_rows = min(df.groupby('topic').size())
df_downsample  = df.groupby('topic', group_keys=False).apply(
    lambda df: df.sample(min_rows))
print(df_downsample.groupby('topic').size())

topic
cultural     11297
economics    11297
politics     11297
sports       11297
dtype: int64


# Preparation and Feature Extraction

Split the dataframe to two parts for training (80%) and testing (20%). (5 points)

In [7]:
train_df, test_df = train_test_split(df_downsample, test_size=0.2, random_state=1)

In this part, we will extract features from the textual data using a very basic method.

Consider the following sentence:
```
Sometimes a dog can run faster than a cat.
```
In the above sentence, the "tokens" are:

```
"sometimes", "a", "dog", "can", "run", "faster", "than", "a", "cat"
```

Find the tokens of each news article in the "training dataframe" by splitting it based on the occurence of the space character (5 points). For instance, the following sentence:
```
او به میدان رفت
```
gets converted to these tokens:
```
["او","به","میدان","رفت"]
```

In [8]:
from tqdm import tqdm

In [9]:
tqdm.pandas()
train_df['tokens'] = train_df.progress_apply(lambda x: x['text'].split(), axis=1)
train_df

100%|██████████| 36150/36150 [00:04<00:00, 8109.45it/s]


,topic,text,tokens
116060,sports,به گزارش ورزش سه رئال مادرید در نیمه اول دیدار...,"[به, گزارش, ورزش, سه, رئال, مادرید, در, نیمه, ..."
156142,sports,به گزارش ورزش سه قرار است کادر پزشکی تیم نساجی...,"[به, گزارش, ورزش, سه, قرار, است, کادر, پزشکی, ..."
161348,cultural,به گزارش ایسنا # # در مراسم معرفی مدیر مرکز اس...,"[به, گزارش, ایسنا, #, #, در, مراسم, معرفی, مدی..."
72689,politics,دولت # اوباما رئیس‌جمهور آمریکا برای آزادی چها...,"[دولت, #, اوباما, رئیس‌جمهور, آمریکا, برای, آز..."
14315,politics,تخصیص کارت اعتباری آثار مثبتی به همراه خواهد د...,"[تخصیص, کارت, اعتباری, آثار, مثبتی, به, همراه,..."
...,...,...,...
111313,sports,به گزارش ورزش سه لیونل مسی بیش از یک دهه است ک...,"[به, گزارش, ورزش, سه, لیونل, مسی, بیش, از, یک,..."
56341,politics,سخنرانی معاون # رئیس‌جمهور در یزد با # روبه‌رو...,"[سخنرانی, معاون, #, رئیس‌جمهور, در, یزد, با, #..."
143529,cultural,مدیر کمیته بین‌الملل # نمایشگاه بین‌المللی کتا...,"[مدیر, کمیته, بین‌الملل, #, نمایشگاه, بین‌المل..."
96024,economics,به گزارش ایسنا تحقق برنامه‌های مرتبط با صادرات...,"[به, گزارش, ایسنا, تحقق, برنامه‌های, مرتبط, با..."


Count the tokens and list the top 5 frequent ones (5 points)

In [10]:
df_all_tokens  = train_df.groupby('topic', group_keys=False).apply(
    lambda train_df: train_df['tokens'])

In [11]:
from itertools import chain

In [12]:
tokens = chain.from_iterable(train_df['tokens'])
counts = pd.Series(tokens).value_counts()
counts.head(5)

#     1633388
و      881451
در     748542
به     585415
از     439101
dtype: int64

Consider the following sentence:
```
تیم فوتبال بارسلونا امشب با بایرن مونیخ دیدار می‌کند.
```
In the above sentence, we do not need the word `با` to determine that the topic is "sports". This word doesn't have any impact on the category this sentence belongs to. 
This word and similar ones are called "stopwords". A list of stopwords for the Persian language can be found in this text file:
https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

Use it to remove all tokens that are actually stopwords.
Also, remove the `#` token. (10 points)

In [13]:
!wget -O /content/stopwords.txt https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt

--2023-05-11 06:47:38--  https://github.com/ziaa/Persian-stopwords-collection/raw/master/Stopwords/Savoy/persianST.txt
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt [following]
--2023-05-11 06:47:39--  https://raw.githubusercontent.com/ziaa/Persian-stopwords-collection/master/Stopwords/Savoy/persianST.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2942 (2.9K) [text/plain]
Saving to: ‘/content/stopwords.txt’

/content/stopwords. 100%[===================>]   2.87K  --.-KB/s    in 0s      

2023-05-11 06:47:39 (26.3 MB/s)

In [14]:
with open('stopwords.txt', 'r') as f:
  lines = f.readlines()
stop_words = [line.rstrip('\n') for line in lines]
stop_words.append('#')
stop_words = set(stop_words)
stop_words

{'#',
 'آباد',
 'آره',
 'آری',
 'آمد',
 'آمده',
 'آن',
 'آنان',
 'آنجا',
 'آنكه',
 'آنها',
 'آنچه',
 'آورد',
 'آورده',
 'آيد',
 'آیا',
 'اثرِ',
 'از',
 'است',
 'استفاده',
 'اش',
 'اكنون',
 'البته',
 'البتّه',
 'ام',
 'اما',
 'امروز',
 'امسال',
 'اند',
 'انکه',
 'او',
 'اول',
 'اي',
 'ايشان',
 'ايم',
 'اين',
 'اينكه',
 'اگر',
 'با',
 'بار',
 'بارة',
 'باره',
 'باشد',
 'باشند',
 'باشيم',
 'بالا',
 'بالایِ',
 'بايد',
 'بدون',
 'بر',
 'برابرِ',
 'براساس',
 'براي',
 'برایِ',
 'برخوردار',
 'برخي',
 'برداري',
 'بروز',
 'بسيار',
 'بسياري',
 'بعد',
 'بعری',
 'بعضي',
 'بلكه',
 'بله',
 'بلکه',
 'بلی',
 'بنابراين',
 'بندي',
 'به',
 'بهترين',
 'بود',
 'بودن',
 'بودند',
 'بوده',
 'بي',
 'بيست',
 'بيش',
 'بيشتر',
 'بيشتري',
 'بين',
 'بی',
 'بیرونِ',
 'تا',
 'تازه',
 'تاكنون',
 'تان',
 'تحت',
 'تر',
 'ترين',
 'تمام',
 'تمامي',
 'تنها',
 'تواند',
 'توانند',
 'توسط',
 'تولِ',
 'تویِ',
 'جا',
 'جاي',
 'جايي',
 'جدا',
 'جديد',
 'جريان',
 'جز',
 'جلوگيري',
 'جلویِ',
 'حتي',
 'حدودِ',
 'حق',
 'خارجِ',
 'خدم

In [15]:
tqdm.pandas()
train_df['new_tokens'] = train_df.progress_apply(
    lambda x: [y for y in x['tokens'] if y not in stop_words], axis=1)
train_df

100%|██████████| 36150/36150 [00:02<00:00, 14441.17it/s]


,topic,text,tokens,new_tokens
116060,sports,به گزارش ورزش سه رئال مادرید در نیمه اول دیدار...,"[به, گزارش, ورزش, سه, رئال, مادرید, در, نیمه, ...","[گزارش, ورزش, سه, رئال, مادرید, نیمه, دیدار, ب..."
156142,sports,به گزارش ورزش سه قرار است کادر پزشکی تیم نساجی...,"[به, گزارش, ورزش, سه, قرار, است, کادر, پزشکی, ...","[گزارش, ورزش, سه, قرار, کادر, پزشکی, تیم, نساج..."
161348,cultural,به گزارش ایسنا # # در مراسم معرفی مدیر مرکز اس...,"[به, گزارش, ایسنا, #, #, در, مراسم, معرفی, مدی...","[گزارش, ایسنا, مراسم, معرفی, مدیر, مرکز, اسناد..."
72689,politics,دولت # اوباما رئیس‌جمهور آمریکا برای آزادی چها...,"[دولت, #, اوباما, رئیس‌جمهور, آمریکا, برای, آز...","[دولت, اوباما, رئیس‌جمهور, آمریکا, برای, آزادی..."
14315,politics,تخصیص کارت اعتباری آثار مثبتی به همراه خواهد د...,"[تخصیص, کارت, اعتباری, آثار, مثبتی, به, همراه,...","[تخصیص, کارت, اعتباری, آثار, مثبتی, همراه, می‌..."
...,...,...,...,...
111313,sports,به گزارش ورزش سه لیونل مسی بیش از یک دهه است ک...,"[به, گزارش, ورزش, سه, لیونل, مسی, بیش, از, یک,...","[گزارش, ورزش, سه, لیونل, مسی, بیش, دهه, ستاره,..."
56341,politics,سخنرانی معاون # رئیس‌جمهور در یزد با # روبه‌رو...,"[سخنرانی, معاون, #, رئیس‌جمهور, در, یزد, با, #...","[سخنرانی, معاون, رئیس‌جمهور, یزد, روبه‌رو, روز..."
143529,cultural,مدیر کمیته بین‌الملل # نمایشگاه بین‌المللی کتا...,"[مدیر, کمیته, بین‌الملل, #, نمایشگاه, بین‌المل...","[مدیر, کمیته, بین‌الملل, نمایشگاه, بین‌المللی,..."
96024,economics,به گزارش ایسنا تحقق برنامه‌های مرتبط با صادرات...,"[به, گزارش, ایسنا, تحقق, برنامه‌های, مرتبط, با...","[گزارش, ایسنا, تحقق, برنامه‌های, مرتبط, صادرات..."


Make a list of the remaining unique tokens (5 points)

In [16]:
new_tokens = chain.from_iterable(train_df['new_tokens'])
unique_tokens_set = set(new_tokens)
unique_tokens = list(unique_tokens_set)
unique_tokens

['تثبیت',
 'رفتارهای',
 'توانست',
 'تمرینی',
 'کیارستمی',
 'موقعیت',
 'می\u200cشوم',
 'مرتکب',
 'شرکت',
 'می\u200cکردند',
 'فر',
 'بخش\u200cهایی',
 'بانکی',
 'جسمانی',
 'کاهش',
 'سال\u200cهای',
 'مهم\u200cتر',
 'اعزامی',
 'می\u200cافتد',
 'کلاس',
 'آوری',
 'مقام',
 'بازیهای',
 'مد',
 'محمدحسین',
 'یارانه\u200cها',
 'شرایط',
 'اختصاصی',
 'خرمشهر',
 'ناشر',
 'تسلط',
 'خداحافظی',
 'وزارتخانه',
 'بیمارستان',
 'حداکثری',
 'شهرستان\u200cها',
 'ری',
 'بیان',
 'دلار',
 'کریمی',
 'برجام',
 'شیرین',
 'آشکار',
 'بیاوریم',
 'ارزشی',
 'چالش',
 'سقوط',
 'بردن',
 'تاثیرگذاری',
 'مسئله',
 'تولیدکنندگان',
 'تکمیل',
 'روندی',
 'دقت',
 'درعین\u200cحال',
 'کاروان',
 'شاخه',
 'پنهان',
 'کفش',
 'علت',
 'ازجمله',
 'کردستان',
 'جهانبخش',
 'مقدماتی',
 'محمدی',
 'منتظری',
 'هستیم',
 'خواسته\u200cهای',
 'والنسیا',
 'ستاره\u200cهای',
 'عامل',
 'مقدار',
 'یکشنبه',
 'بغداد',
 'مهم\u200cترین',
 'داری',
 'جداگانه',
 'باشید',
 'پایانی',
 'نقدینگی',
 'کرده\u200cایم',
 'فضایی',
 'فعلی',
 'انتظامی',
 'شخصیت\u200cهای',
 '

Count the number of the occurences of each token in each document. This way, you can make a numpy array to represent each document. Call this new array `x_train`. (10 points)

Example:

Unique tokens in all documents: 
```
0: hen
1: sometimes
2: a
3: the
4: dog
5: runs
6: faster
7: than
8: cat
9: bird
```

Current Document: `sometimes a dog runs faster than a cat`

The representation of the document:

```
[0, 1, 2, 0, 1, 1, 1, 1, 1, 0]
```

The meaning of this representation:

```
0: the document doesn't contain "hen"
1: the document contains 1 "sometimes"
2: the document contains 2 "a"
0: the document doesn't contain "the"
1: the document contains 1 "dog"
1: the document contains 1 "runs"
1: the document contains 1 "faster"
1: the document contains 1 "than"
1: the document contains 1 "cat"
0: the document doesn't contain "bird"
``` 

In [17]:
def hash_token(token):
  x = 0
  for c in token:
    x += ord(c)
  x = x * len(token)
  return x % len(unique_tokens)

In [18]:
def count_tokens(tokens):
  count_array = [0 for _ in unique_tokens]
  for token in tokens:
    if token in unique_tokens_set:
      count_array[hash_token(token)] += 1
  return count_array

In [19]:
tqdm.pandas()
x_train = train_df.progress_apply(
    lambda x: count_tokens(x.new_tokens), axis=1)
x_train

100%|██████████| 36150/36150 [00:28<00:00, 1274.24it/s]


116060    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
156142    [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
161348    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
72689     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
14315     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                                ...                        
111313    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
56341     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
143529    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
96024     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
57416     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
Length: 36150, dtype: object

Make another numpy array by converting the topics associated with the training news to numbers. For instance, if the topics are: `sports`, `economics`, `politics`, and `cultural`, convert them to `0` to `3`. Call this new array `y_train`. (5 points)

In [20]:
def convert_topics(topic):
  topics = ['sports', 'economics', 'politics', 'cultural']
  return topics.index(topic)

In [21]:
tqdm.pandas()
y_train = train_df.progress_apply(
    lambda x: convert_topics(x.topic), axis=1)
y_train = y_train.to_numpy()
y_train

100%|██████████| 36150/36150 [00:00<00:00, 82562.75it/s]


array([0, 0, 3, ..., 3, 1, 2])

Using the unique tokens you found in the training news, represent the testing news as another numpy array called `x_test`. (5 points)

Note: Do not forget to remove stopwords and the `#` token.

In [22]:
tqdm.pandas()
test_df['tokens'] = test_df.progress_apply(lambda x: x['text'].split(), axis=1)
test_df

100%|██████████| 9038/9038 [00:01<00:00, 7584.16it/s]


,topic,text,tokens
154193,sports,به گزارش ورزش سه سرانجام پس از مدت‌ها # زنی با...,"[به, گزارش, ورزش, سه, سرانجام, پس, از, مدت‌ها,..."
108734,sports,به گزارش ورزش سه دو روز قبل بود که فدراسیون کش...,"[به, گزارش, ورزش, سه, دو, روز, قبل, بود, که, ف..."
115021,politics,معاون سیاسی وزیر کشور دولت برای فعالان سیاسی د...,"[معاون, سیاسی, وزیر, کشور, دولت, برای, فعالان,..."
36430,economics,رییس کمیسیون واردات اتاق بازرگانی ایران شرایط ...,"[رییس, کمیسیون, واردات, اتاق, بازرگانی, ایران,..."
75016,sports,به گزارش ورزش سه # شفر و آندره آ استراماچونی ب...,"[به, گزارش, ورزش, سه, #, شفر, و, آندره, آ, است..."
...,...,...,...
33629,cultural,وزیر فرهنگ و ارشاد اسلامی در آیین افتتاح # باغ...,"[وزیر, فرهنگ, و, ارشاد, اسلامی, در, آیین, افتت..."
139909,cultural,به گزارش ایسنا متقاضیان دریافت این # پژوهشی با...,"[به, گزارش, ایسنا, متقاضیان, دریافت, این, #, پ..."
28168,cultural,به گزارش ایسنا در معرفی مرکز اسناد انقلاب اسلا...,"[به, گزارش, ایسنا, در, معرفی, مرکز, اسناد, انق..."
92903,sports,به گزارش ورزش سه بعد از این بازی مالک باشگاه م...,"[به, گزارش, ورزش, سه, بعد, از, این, بازی, مالک..."


In [23]:
tqdm.pandas()
test_df['new_tokens'] = test_df.progress_apply(
    lambda x: [y for y in x['tokens'] if y not in stop_words], axis=1)
test_df

100%|██████████| 9038/9038 [00:00<00:00, 9953.75it/s] 


,topic,text,tokens,new_tokens
154193,sports,به گزارش ورزش سه سرانجام پس از مدت‌ها # زنی با...,"[به, گزارش, ورزش, سه, سرانجام, پس, از, مدت‌ها,...","[گزارش, ورزش, سه, سرانجام, مدت‌ها, زنی, باشگاه..."
108734,sports,به گزارش ورزش سه دو روز قبل بود که فدراسیون کش...,"[به, گزارش, ورزش, سه, دو, روز, قبل, بود, که, ف...","[گزارش, ورزش, سه, روز, فدراسیون, کشتی, خبری, م..."
115021,politics,معاون سیاسی وزیر کشور دولت برای فعالان سیاسی د...,"[معاون, سیاسی, وزیر, کشور, دولت, برای, فعالان,...","[معاون, سیاسی, وزیر, کشور, دولت, برای, فعالان,..."
36430,economics,رییس کمیسیون واردات اتاق بازرگانی ایران شرایط ...,"[رییس, کمیسیون, واردات, اتاق, بازرگانی, ایران,...","[رییس, کمیسیون, واردات, اتاق, بازرگانی, ایران,..."
75016,sports,به گزارش ورزش سه # شفر و آندره آ استراماچونی ب...,"[به, گزارش, ورزش, سه, #, شفر, و, آندره, آ, است...","[گزارش, ورزش, سه, شفر, آندره, آ, استراماچونی, ..."
...,...,...,...,...
33629,cultural,وزیر فرهنگ و ارشاد اسلامی در آیین افتتاح # باغ...,"[وزیر, فرهنگ, و, ارشاد, اسلامی, در, آیین, افتت...","[وزیر, فرهنگ, ارشاد, اسلامی, آیین, افتتاح, باغ..."
139909,cultural,به گزارش ایسنا متقاضیان دریافت این # پژوهشی با...,"[به, گزارش, ایسنا, متقاضیان, دریافت, این, #, پ...","[گزارش, ایسنا, متقاضیان, دریافت, این, پژوهشی, ..."
28168,cultural,به گزارش ایسنا در معرفی مرکز اسناد انقلاب اسلا...,"[به, گزارش, ایسنا, در, معرفی, مرکز, اسناد, انق...","[گزارش, ایسنا, معرفی, مرکز, اسناد, انقلاب, اسل..."
92903,sports,به گزارش ورزش سه بعد از این بازی مالک باشگاه م...,"[به, گزارش, ورزش, سه, بعد, از, این, بازی, مالک...","[گزارش, ورزش, سه, این, بازی, مالک, باشگاه, ملو..."


In [24]:
tqdm.pandas()
x_test = test_df.progress_apply(
    lambda x: count_tokens(x.new_tokens), axis=1)
x_test

100%|██████████| 9038/9038 [00:08<00:00, 1111.14it/s]


154193    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
108734    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...
115021    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
36430     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
75016     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                                ...                        
33629     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
139909    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
28168     [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
92903     [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...
168920    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Length: 9038, dtype: object

Convert the topics associated with the testing news to their equivalent numbers (as before), and save the result as a numpy array called `y_test`. (5 points)

In [25]:
tqdm.pandas()
y_test = test_df.progress_apply(
    lambda x: convert_topics(x.topic), axis=1)
y_test

100%|██████████| 9038/9038 [00:00<00:00, 88974.29it/s]


154193    0
108734    0
115021    2
36430     1
75016     0
         ..
33629     3
139909    3
28168     3
92903     0
168920    2
Length: 9038, dtype: int64

# Model Training and Evaluation

## Using sklearn (10 points)

Train a Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`) using `sklearn`.

In [26]:
X = []
for x in tqdm(x_train):
  X.append(x)
x_train = np.array(X)

100%|██████████| 36150/36150 [00:00<00:00, 1986506.60it/s]


In [27]:
mnb_model_sklearn = MultinomialNB(force_alpha=True)
mnb_model_sklearn.fit(x_train, y_train)

MultinomialNB(force_alpha=True)

Get the predictions of the model for the testing news (`x_test`).

In [28]:
X = []
for x in x_test:
  X.append(x)
x_test = np.array(X)

In [29]:
y_predict_sklearn = mnb_model_sklearn.predict(x_test)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [30]:
print(classification_report(y_test, y_predict_sklearn))

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      2285
           1       0.89      0.92      0.91      2187
           2       0.93      0.90      0.92      2239
           3       0.93      0.94      0.94      2327

    accuracy                           0.94      9038
   macro avg       0.94      0.94      0.94      9038
weighted avg       0.94      0.94      0.94      9038



## Using your own code (15 points)

In [31]:
from math import sqrt, pi, exp

In [78]:
class Naive_Bayes:
    """
    Fits it on data, then uses predict to get results.
    """

    def __init__(self):
        """
        YOUR CODE (IF NECESSARY)
        """
        self._classes = None
        self._mean = None
        self._var = None
        self._priors = None
        self._feature_counts = None
    
    def fit(self, X, y):
        """Fit the training data
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Training samples
        y : array-like, shape = [n_samples, n_target_values]
            Target values
        Returns
        -------
        self : object
        """

        n_samples, n_features = X.shape
        """
        YOUR CODE
        """
        self._classes = np.unique(y)
        n_classes = len(self._classes)

        self._mean = np.zeros((n_classes, n_features), dtype=np.float64)
        self._var = np.zeros((n_classes, n_features), dtype=np.float64)
        self._priors = np.zeros(n_classes, dtype=np.float64)
        self._feature_counts = np.zeros((n_classes, n_features), dtype=int)

        for idx, c in enumerate(self._classes):
          X_c = X[y==c]
          self._mean[c,:] = X_c.mean(axis=0)
          self._var[c,:] = X_c.var(axis=0)
          self._priors[c] = X_c.shape[0] / float(n_samples)
          self._feature_counts[idx] = np.sum(X_c, axis=0)

        self._feature_probs = (
            self._feature_counts + 1) / (
                np.sum(self._feature_counts, axis=1)[:, np.newaxis] + n_features)
    
    def predict(self, X):
        """ Predicts the value after the model has been trained.
        Parameters
        ----------
        x : array-like, shape = [n_samples, n_features]
            Test samples
        Returns
        -------
        Predicted value
        """
        
        y_pred = [self._predict(x) for x in X]
        """
        YOUR CODE
        """
        return y_pred

    def _predict(self, x):
        probs = self._feature_probs[:, x > 0]
        log_probs = np.log(probs)
        log_probs += np.log(self._priors[:, np.newaxis])
        return self._classes[np.argmax(np.sum(log_probs, axis=1))]

Train your own Multinomial Naive Bayes Model on the training news (`x_train` and `y_train`).

In [79]:
nb_model = Naive_Bayes()
nb_model.fit(x_train, y_train)

Get the predictions of your model for the testing news (`x_test`).

In [80]:
y_pred_scratch = nb_model.predict(x_test)

Print the classification report containing "precision", "recall", and "f1-score" for each class, and their averages. (you can use `sklearn` for this part)

In [81]:
print(classification_report(y_test, y_pred_scratch))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98      2285
           1       0.88      0.90      0.89      2187
           2       0.89      0.92      0.90      2239
           3       0.93      0.91      0.92      2327

    accuracy                           0.92      9038
   macro avg       0.92      0.92      0.92      9038
weighted avg       0.92      0.92      0.92      9038

